### Programming for Science and Finance

*Prof. Götz Pfeiffer, School of Mathematical and Statistical Sciences, University of Galway*

# Notebook 6: Scientific Programming II

This notebook accompanies **Part II**. You will:

* extend your understanding of **array operations** to build practical filtering and analysis tools for scientific and image data;
* learn how **discrete convolution** combines signals or images with kernels to smooth, sharpen, or detect structure;
* distinguish between **convolution** and **cross-correlation**, and see how each is used in signal and image processing;
* apply **moving-average filters** as a simple form of data smoothing in one dimension;
* generate and use **Gaussian kernels** for controlled blurring, both in 1D and 2D;
* implement **2D convolution** to process grayscale and colour images efficiently;
* and apply these techniques to **edge detection**, showing how local contrast reveals boundaries and features.

By the end of this notebook, you will understand how convolution underpins a wide range of operations in science, finance, and computer vision — and how NumPy enables these ideas to be expressed concisely and computed efficiently.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

## Task 1. Convolutions.

**Recall** how to **multiply polynomials**:
$$
\biggl(\sum_{i=0}^{m-1} a_i x^i\biggr) \cdot  \biggl(\sum_{j=0}^{n-1} b_j x^j\biggr) = \biggl(\sum_{k=0}^{(m-1+n-1} c_k x^k\biggr),
$$
where 
$$
c_k = \sum_{i+j = k} a_i b_j = \sum_i a_i b_{k-i}.
$$

**Example.** $$(1 + 2x + 3x^2)(4 + 5x + 6x^2) = 4 + 13x + 28x^2 + 27x^3 + 18x^4$$. 

The **convolution** of lists $a = (a_0, \dots, a_{m-1})$ (of length $m$) and $b = (b_0, \dots, b_{n-1})$ (of length $n$) is  the list $a * b = c = (c_0, \dots, c_{n-1+m-1})$ (of length $m+n-1$), where 
$$
c_k = \sum_{i+j = k} a_i b_j = \sum_i a_i b_{k-i}
$$
as above.

**Example.** $$(1, 2, 3) * (4, 5, 6) = (4, 13, 28, 27, 18)$$. 

Numpy can compute convolutions with `np.convolve`.

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]
np.convolve(a, b)

It should not be too difficult to write our own python function for this.
However, if the first attempt is
```python
def convolve0(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        for i in range(m):
            j = k - i
            c[k] += a[i] * b[j]
    return c
```
then it will fail with an `IndexError` because some `j` eventually is as big as `len(b)`.
So perhaps we safeguard against this by testing whether `j` is a valid index for `b`:

In [ ]:
def convolve1(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        for i in range(m):
            j = k - i
            if 0 <= j < n:
                c[k] += a[i] * b[j]
    return c

convolve1(a, b)

This works, obviously.  But then it is an example of **very bad programming**.
A test at the inner level of two nested loops can be very expensive, since it is performed at any single iteration.
Even worse, in this case it is already known whether `j = k - i` will be a good index for `b` when `i` is chosen!
So there would be no need for a test if only more care had been taken in the choice of `i`. 

Let's unravel the situation.  In terms of $i$, the condition on $j = k - i$ reads
$$
0 \leq k - i < n
$$
Subtracting $k$ throughout, and then taking negatives, turns this into
$$
k \geq i > k-n
$$
or
$$
k - n + 1 \leq i < k + 1
$$
Since, at the same time, we need $0 \leq i < m$, we overall have
$$
\max(0, k-n+1) \leq i < \min(m, k+1)
$$
and we should choose the range for $i$ accordingly.

In [ ]:
def convolve2(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        for i in range(max(0, k-n+1), min(m, k+1)):
            c[k] += a[i] * b[k - i]
    return c

convolve2(a, b)

Now, perhaps **list comprehension** can simplify the code ...

In [ ]:
def convolve3(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        c[k] = sum(a[i] * b[k-i] for i in range(max(0, k-n+1), min(m, k+1)))
    return c

convolve3(a, b)

... and **vectorization** improves it even further, if we make sure that the input lists `a` and `b` are arrays.

In [ ]:
a = np.array(a)
b = np.array(b)

In [ ]:
def convolve4(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        iii = np.arange(max(0, k-n+1), min(m, k+1))
        c[k] = sum(a[iii] * b[k-iii])
    return c

convolve4(a, b)

Finally, we use `np.sum` instead of `sum`.

In [ ]:
def convolve5(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m + n - 1)
    for k in range(m + n - 1):
        iii = np.arange(max(0, k-n+1), min(m, k+1))
        c[k] = np.sum(a[iii] * b[k-iii])
    return c

convolve5(a, b)

---
**Exercises.**

1. Explain in your own words the differences between the 5 implementations of convolution.

3. What is `a[iii]` when `iii` is a list of indices?  Does this kind of indexing work in plain python?

2. What is the difference between using `sum` and using `np.sum`?

4. Measure the time it takes for each of the five versions to compute a convolution:
   ```python
   rng = np.random.default_rng()
   a, b = rng.random((2, 900))

   %timeit c1 = convolve1(a, b)
   %timeit c2 = convolve2(a, b)
   %timeit c3 = convolve3(a, b)
   %timeit c4 = convolve4(a, b)
   %timeit c5 = convolve5(a, b)
   ```
   Does it make difference, in a sufficiently big example?

5. In the final version, `convolve5`, replace `np.sum(...)` by `(...).sum()` and call the resulting function `convolve6`:
   ```python
   def convolve6(a, b):
       m, n = len(a), len(b)
       c = np.zeros(m + n - 1)
       for k in range(m + n - 1):
           iii = np.arange(max(0, k-n+1), min(m, k+1))
           c[k] = (a[iii] * b[k-iii]).sum()
       return c
   ```
Does that give the same results?  Is it faster or slower?

6. Include `np.convolve` in the performance comparison.

7. Check that all versions produce the same numerical results:
   ```python
   funcs = [convolve1, convolve2, convolve3, convolve4, convolve5, convolve6]
   values = np.convolve(a, b)
   for f in funcs:
       assert np.allclose(f(a, b), values)
   ```

---

## Task 2. Cross Correlation

We will apply convolutions to computed (weighted) moving averages.
For this, we change our point of view on convolutions $a * b$ slightly.
In particular, we truncate the resulting list to have the **same length** as the longer of $a$ and $b$.
`np.convolve` does this when supplied with an additional argument `"same"`.

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]
np.convolve(a, b, 'same')

More specifically, we will treat lists $a$ and $b$ differently:

* We assume that $a$ is long and $b$ is short, also technically that is not necessary.
* But we do assume that $b$ has odd length $n = 2j+1$ (and that $a$ has length $m$).
* We also assume that $b$ is symmetric (palindromic) so it needs not be reverted.
* The resulting list $c$ should be a modification of $a$, in the sense that
  $$c_k = \sum_{i = -j}^{j} a_{k+i} b_i,$$
  so that each $a_k$ is replaced by the $b$-weighted sum `a[k-j:k+j+1] * b` of itself
  and its $j$ neighbors to the left, and to the right. 

This will further simplify the code, after the $2j$ non-existing elements `a[-j:0]` (for $k = 0$) and `a[m:m+j]` (for $k = m-1$) of the list `a` have been taken care of by padding `a` with $0$ s. 

```python
def correlate0(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m)
    for k in range(m):
        iii = k - n//2 + np.arange(n)
        c[k] = sum(a[iii] * b[0:n])
    return c
```
won't work.  But if we replace `a` with `np.pad(a, n//2)` (appending `n//2` zeros on either side) then this one does:

In [ ]:
def correlate1(a, b):
    m, n = len(a), len(b)
    c = np.zeros(m)
    a = np.pad(a, n//2)
    for k in range(m):
        c[k] = np.sum(a[k:k+n] * b)
    return c

In [ ]:
correlate1(a, np.flip(b))

And this code, again, can be improved by **list comprehension**:

In [ ]:
def correlate_1d(a, ker):
    m, n = len(a), len(ker)
    a = np.pad(a, n//2)
    return np.array([(a[k:k+n] * ker).sum() for k in range(m)])

In [ ]:
correlate_1d(a, np.flip(b))

---
**Exercises.**

1. Suppose that $a$ has length $m$ and that $b$ has odd length $n = 2n' + 1$.  
Let $a'$ be a copy of $a$, padded with $n'$ zeros on each side, i.e., $a'_i = a_{i - n'}$.  
Let $b'$ be $b$ reversed, i.e., $b'_j = b_{n-1-j}$.  Define
$$
c'_k = \sum_{j}  a'_{k+j} b'_j.
$$
Show that $c'_k = c_{k + n'}$, where $c = a * b$ is the **convolution** of $a$ and $b$.

1. How does this formula relate the implementation of `correlate_1d` to the convolution product?

2. Write a version of `correlate_1d` that uses **reflection padding** (extend edge values instead of zero padding).

---

## Task 3.  Moving Averages

Convolutions, or correlations, can be used to compute **moving averages** easily.  
This moving average filter is an example of a **low-pass filter**, which suppresses rapid fluctuations (high frequencies) in the signal.

In [ ]:
w = 3
ker = np.ones(w)/w
correlate_1d(np.arange(10), ker)

We illustrate this concept with a more striking application.
For this, let's start with the plot of a sine curve, or more precisely, one wave of it.

In [ ]:
xxx = np.linspace(0, 2*math.pi, 50)
yyy = np.sin(xxx)  # vectorized sin
plt.plot(xxx, yyy)

Now we add some noise, normally distributed

In [ ]:
rng = np.random.default_rng()
noise = rng.normal(scale =0.4, size=xxx.shape)
plt.scatter(xxx, noise)

Let's see whether these values actullay form a Bell curve.

In [ ]:
plt.hist(noise, bins=7)

Now we add the noise to the sine curve.

In [ ]:
data = yyy + noise
plt.plot(xxx, data)

We'll try and get the sine curve back by avering over, say, $w= 11$ values.

In [ ]:
w = 11
kernel = np.ones(w)/w
plt.plot(xxx, correlate_1d(data, kernel))

---
**Exercises.**

1. What is the effect of convolving the noisy sine curve with a longer, say `w = 21`, constant kernel?

2. What is the effect of a kernel that puts more weight on the current `x`-value, like `k= np.array([1, 2, 1])/4`?

3. Use `np.repeat(k, 5)` to produce a "stretched" version of the previous kernel. What effect would that have?

---

## Task 4. Gaussian Kernels

The continuous **Gaussian function** (for expected value $\mu = 0$) is
$$
G(x)=\frac1{\sqrt{2\pi} \sigma} \exp(-\frac12 \frac{x^2}{\sigma^2})
$$
In discrete form, we sample it symmetrically around zero — say, at integer points 
$−r,\dots,r$,
where 
$r = ⌊3\sigma⌋$ (three standard deviations is a good truncation).

In [ ]:
def gaussian_kernel_1d(sigma):
    radius = int(3 * sigma)
    x = np.arange(-radius, radius + 1)
    g = np.exp(-0.5 * (x / sigma)**2)
    return g/g.sum()

In [ ]:
gauss = gaussian_kernel_1d(3)
print(gauss, gauss.sum())
plt.plot(xxx, correlate_1d(data, gauss))

Let's apply a Gaussian kernel to a **time series** of **stock prices**.
We `import` and use the `yfinance` package to access historical stock price data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

The data can be downloaded and accessed as follows.  
Here, `data` is a `pandas` data frame.
And its column `data["Close"]` is a `Series` object, that can easily be plotted.

In [ ]:
tick = yf.Ticker("TSLA")
data = tick.history(period="ytd")
close = data["Close"]
close.plot()

In [ ]:
type(close.values)

In [ ]:
close.values.shape

In [ ]:
val = close.values
kernel = gaussian_kernel_1d(12)
con = correlate_1d(val, kernel)
pd.Series(con, index = close.index).plot()

---
**Exercises.**

1. What is the `type` of `data` and `close`?  What is the type (and shape) of `data.values` and `close.values`?

1. Why does the smoothed time series tend to $0$, at the beginning and at the end of the time interval?

2. Use `correlate_1d` to compute a 5-day rolling weighted average of Tesla’s closing prices.

1. Compute straight moving averages of the above time series, with a kernel `np.ones(w)/w` for `w = 25`, say. 

1. Modify the time period of the time series (after studying the documentation of `Ticker.history`).

2. Plot one year's worth of the `"META"` ticker, before and after applying convolution with a Gaussian kernel.
---

## Task 5.  2D Convolution

In [ ]:
from PIL import Image


* Next up we'll look at convolutions of functions of two variables.
* The use case for us is pixels in images, so we will only deal with the discrete case.
* Here we let $f$ and $g$ be functions $f,g : \mathbb{Z}^2 \rightarrow \mathbb{R}$, and define their convolution to be
  $$
  (f\ast g)(m,n) = \sum_{i=-\infty}^{+\infty} \sum_{j=-\infty}^{+\infty} f(i,j)g(m-i,n-j)
  $$

In [ ]:
def correlate_2d(A, ker):
    m, n = A.shape
    p, q = ker.shape
    A = np.pad(A, ((p//2, p//2), (q//2, q//2)))
    return np.array([[
            np.sum(A[i:i+p, j:j+q] * ker) for j in range(n)
        ] for i in range(m)
    ])

#### An example image

* Let's generate an image of size 100 by 100 pixels, with two circles drawn on it.
* The low resolution is intentional, because it allows us to better see the blurring effect of applying a convolution.

In [ ]:
circles = np.zeros((100,100,3), dtype='uint8')
for i in range(circles.shape[0]):
    for j in range(circles.shape[1]):
        if (i-20)**2 + (j-15)**2 <= 10**2:
            # a red circle with centre (20,15) and radius 10
            circles[i,j,0]=255
        if (i-60)**2 + (j-55)**2 <= 25**2:
            # a blue circle with centre (60,55) and radius 25
            circles[i,j,2] = 255

Image.fromarray(circles)

* Using matplotlib, we can zoom in a bit.

In [ ]:
plt.imshow(circles[20:50,20:50])

* We can see that the image is rather blocky, so we might want to smoothen the circles by blurring.
* To this end, we will convolve the image with a $3 \times 3$ matrix in such a way that the intensity of a pixel will be the average of the  original intensity in the pixel and that of its neighbouring pixels.

In [ ]:
kernel = np.ones((3,3))/9
kernel

* Recall how a digital image is really 3 matrices: one for each color channel R, G and B.
* We apply the convolution ot each of the three matrices in turn.
* Finally, we combine the three resulting matrices into a 3D tensor with `np.stack` along `axis = 2`.
* This gives a digital image that can be displayed.

In [ ]:
def correlate_rgb(img, ker):
    img = img.astype(float)
    channels = [correlate_2d(img[..., c], ker) for c in range(3)]
    return np.stack(channels, axis=2)

In [ ]:
blur = np.clip(correlate_rgb(circles, kernel), 0, 255).astype(np.uint8)
Image.fromarray(blur)

In [ ]:
f, axarray = plt.subplots(1,2, figsize=(16,6))
axarray[0].imshow(circles[20:50,20:50])
axarray[1].imshow(blur[20:50,20:50])

#### 2D Gaussian Kernel

Note how the 2D blur is a matrix product.

In [ ]:
ker = np.ones(3)/3
np.outer(ker, ker)

In the same way, we can create a Gaussion 2D kernel.

In [ ]:
gauss = gaussian_kernel_1d(3)
gauss2 = np.outer(gauss, gauss)
np.sum(gauss2)

In [ ]:
gblur = np.clip(correlate_rgb(circles, gauss2), 0, 225).astype(np.uint8)
Image.fromarray(gblur)

In [ ]:
f, axarray = plt.subplots(1,2, figsize=(16,6))
axarray[0].imshow(blur[20:50,20:50])
axarray[1].imshow(gblur[20:50,20:50])

---
**Exercises.**

1. Build a 2D kernel as matrix product from the 1D kernel `k = np.array([1,2,1])/4` and apply this to the `circles` picture.  How does the result compare to the above convolutions?

2. Apply convolution with the various kernels to the image in the file `"images/long_walk.png"`.

---

## Task 6. Edge Detection

* We will use **convolution**, or rather **correlation**, with a suitable kernel to **detect edges** in a digital image.
* A so-called **Sobel Filter**  uses two $3 \times 3$ kernels $G_x$ and $G_y$, which are convolved with the original image.
* Their purpose is to identify horizontal and vertical changes:
  $$
  G_x = \left[\begin{array}{ccc}
  1 & 0 & -1 \\
  2 & 0 & -2 \\
  1 & 0 & -1
  \end{array}\right],
  \quad
  G_y = \left[\begin{array}{ccc}
  1 & 2 & 1 \\
  0 & 0 & 0 \\
  -1 & -2 & -1
  \end{array}\right],
  $$
* Note how $G_y = G_x^T$.
* Also, how $G_x = (1, 2, 1)^T (1, 0, -1)$ is a product of a **smoothing** filter and a **central difference**.
* As such, convolution with $G_x$ yields the $x$-part of the gradient of the image intensity function, and convolution with $G_y$ yields the $y$-part.
* The overall gradient of a matrix $A$ can thus be found as $\sqrt{((A*G_x)^2 + (A*G_y)^2}$, where the square is meant component-wise (and not as matrix product).

In [ ]:
np.array([[1,2,1]]).T @ np.array([[1,0,-1]])  # matrix product

In numpy, we can compute the matrix product $u^T v$ of two vectors $u$ and $v$ as **outer product**.

In [ ]:
s = np.array([1, 2, 1])  # smoothing
d = np.array([1, 0, -1])  # differentiation
Gx = np.outer(s, d)
Gy = np.outer(d, s)
Gx, Gy

* Apply the two kernels to the 2 circles picture, compute the gradient and display as image.

In [ ]:
dx = correlate_rgb(circles, Gx)
dy = correlate_rgb(circles, Gy)
edges = np.sqrt(dx**2 + dy**2)
plt.imshow(np.clip(edges, 0, 255).astype(np.uint8))

* Wrap all of this up in a function `edge_detect`:

In [ ]:
def edge_detect(img):
    s, d = np.array([[1,2,1], [1,0,-1]])
    Gx, Gy = np.outer(s, d), np.outer(d, s)
    dx, dy = correlate_rgb(img, Gx), correlate_rgb(img, Gy)
    return np.clip(np.sqrt(dx**2 + dy**2), 0, 255).astype(np.uint8)

* And test the function on the `circles` picture.

In [ ]:
Image.fromarray(edge_detect(circles))

---
**Exercises**.

1. Considering the polynomials $(x+1)^2$ and $(x-1)(x+1)$, find decompositions of the vectors $(1,2,1)$ and $(1, 0, -1)$ as **convolution products** of shorter vectors.

1. Using the **matrix product** decompositions of $G_x = (1, 2, 1)^T (1, 0, -1)$, show that, for any matrix $A$, the convoltion $A * G_x$ is the same as the iterated convolution $(A * (1, 2, 1)^T) * (1, 0, -1)$.

3. Apply this edge detection procedure to the image in the file `"images/long_walk.png"`.
---

## Summary


In this notebook you have seen how **convolution and correlation** form the foundation of many techniques in data analysis and image processing.
You learned how to:

* express convolution efficiently in Python using **NumPy arrays** and **vectorized operations**;
* understand the connection between **convolution**, **cross-correlation**, and **moving averages** as tools for smoothing or pattern matching;
* generate **Gaussian kernels** to create smooth, natural blurring filters;
* extend convolution to **two dimensions**, applying it to digital images channel-by-channel;
* and use simple kernels to perform **edge detection**, extracting structure and contrast from raw data.

These operations demonstrate how powerful scientific programming becomes when mathematical ideas are written directly in array form.
You now have the essential tools to design and experiment with your own signal- and image-processing pipelines — the same principles that underlie modern analysis methods in **science, finance, and machine learning**.
